# IBM Streams database sample application
This sample demonstrates creating a Streams Python application to connect to a Db2 database, performing some analytics, and viewing the results.

In this notebook, you'll see examples of how to:
1. [Setup your database connection](#setup)
2. [Create the application](#create)
3. [Submit the application](#submit)
4. [Connect to the running application to view data](#view)

# Overview
**About the sample**

This application simulates data tuples that are inserted as rows into a Db2 database table.

**How it works**
   
The Python application created in this notebook is submitted to the IBM Streams service for execution. Once the application is running in the service, you can connect to it from the notebook to retrieve the results.

<img src="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/04/how-it-works.jpg" alt="How it works">


### Documentation
- [Streams Python development guide](https://ibmstreams.github.io/streamsx.documentation/docs/latest/python/)
- [Streams Python API](https://streamsxtopology.readthedocs.io/)



## <a name="setup"> </a> 1. Setup

### 1.1 Add credentials for the IBM Streams service

In order to submit a Streams application you need to provide the name of the Streams instance.

1. From the navigation menu, click **My instances**.
2. Click the **Provisioned Instances** tab.
3. Update the value of `streams_instance_name` in the cell below according to your Streams instance name.

In [1]:
from icpd_core import icpd_util
streams_instance_name = "cp4d-streams-instance" ## Change this to Streams instance
cfg=icpd_util.get_service_instance_details(name=streams_instance_name)

### 1.2 Optional: Upgrade the `streamsx.database` Python package

Uncomment and run the cell below if you want to upgrade to the latest version of the streamsx.database package.


In [ ]:
#!pip install --user --upgrade streamsx.database

The python packages will be installed in the top of user path.<br/>
If you have problem to get the latest version of python packages you can set the order of python packages manually to user path.<br/>
you can find the user path with this command:<br/>
`
import sys
for e in sys.path:
    print(e)
`

In [ ]:
#import sys
#sys.path.insert(0, '/home/wsuser/.local/lib/python3.6/site-packages')

In [2]:
import streamsx.database as db
import streamsx.topology.context
print("INFO: streamsx package version: " + streamsx.topology.context.__version__)
print("INFO: streamsx.database package version: " + db.__version__)

INFO: streamsx package version: 1.13.14
INFO: streamsx.database package version: 1.4.0


### <a name="credentials"> </a> 1.3 Configure the connection to Db2 Warehouse

We need a Db2 credentials as JSON string to connect to a Db2 database.
This JSON string contains the database credentials `username`, `password` and `jdbcurl`.

To create a Db2 credentials, please perform the following steps:

1. Create a Db2 Warehouse service on IBM cloud.

  you need to have an IBM account to create a Db2 service.

  https://console.bluemix.net/catalog/?search=db2

2. Create a service credential for Db2 service on IBM cloud.
3. Copy the credentials in clipboard.
4. Paste the credentials into Db2 Warehouse credentials prompt below.

If you want to use another Db2 database, you can create a JSON string with the following attributes:

    {
      "username": "your-db-user-name",
      "password": "your-db-password",
      "jdbcurl": "jdbc:db2://your-db2-hostname:50000/your-database-name"
    }

In [3]:
import getpass
db2_service_credentials=getpass.getpass('Db2 Warehouse credentials:')

KeyboardInterrupt: 

In [9]:
import json
    
anotherdb2_service_credentials = {
  "username": "user1018",
  "password": "cRs*B2S#v4#75@Dt",
  "jdbcurl": "jdbc:db2://worker2:31050/BLUDB"
}
anotherdb2_service_credentials_J = json.dumps(anotherdb2_service_credentials)
# store the database credentials in db2credentials
db2credentials = json.loads(anotherdb2_service_credentials_J)

### 1.3.1 Alternative: Use a configured external connection

Perform the steps [Connecting to data source](https://docs-icpdata.mybluemix.net/docs/content/SSQNUZ_current/com.ibm.icpdata.doc/igc/t_connect_data_sources.html) 
and create an external configuration for your Db2 connection.

List the connections with the cell below:

In [10]:
ext_connections = icpd_util.get_connections('external')
print (ext_connections)

[{'class': 'external', 'creator': {}, 'database_name': 'opendata', 'host': '140.110.19.12', 'name': 'epa_postgreSQL', 'port': 5432, 'type': 'PostgreSQL', 'url': 'jdbc:ibmdriver:postgresql://140.110.19.12:5432/opendata'}, {'additional_data': {'bucket_name': 'obj0708'}, 'class': 'external', 'creator': {}, 'name': 'Thomas-Data-f-streams-test', 'type': '_amazons3'}, {'additional_data': {'bucket_name': 's3.twcc.ai'}, 'class': 'external', 'creator': {}, 'name': 'twccs3', 'type': '_amazons3'}, {'class': 'external', 'creator': {}, 'database_name': 'youbike', 'host': '203.145.221.37', 'name': 'twccdb', 'port': 3306, 'type': 'MySQL-CE', 'url': 'jdbc:mysql://203.145.221.37:3306/youbike'}]


Change the connection_name, uncomment and run the cell below 

In [11]:
#connection_name = 'Db2-Ext'
#db2credentials = icpd_util.get_connection(connection_name, conn_class='external')
#print (db2credentials)

# <a name="create"> </a> 2. Create the application
All Streams applications start with a Topology object, so start by creating one:


In [12]:
#Imports
from streamsx.topology.topology import *
from streamsx.topology.context import *
from streamsx.topology.schema import StreamSchema
import streamsx.database as db


# create a Topology object
topo = Topology(name="DatabaseSample")


### How to use the streamsx.database package
The streamsx.database package is the python wrapper for the [streamsx.jdbc](https://ibmstreams.github.io/streamsx.jdbc/doc/spldoc/html) toolkit

To interact with a Db2 database from Streams, you pass a SQL statement to the `streamsx.database.run_statement` function.

`run_statement` is the main function of `streamsx.database` package.

It executes a SQL statement and produces a [`Stream`](https://streamsxtopology.readthedocs.io/en/stable/streamsx.topology.topology.html#stream) of the results.
It needs at least two mandatory parameters, the first one is the input `Stream` and the second parameter is the database credentials in JSON format.

There are 2 ways to execute SQL statements using `db.run_statement`:
- Set the `stream` parameter to a `Stream` containing the statements to execute. This is used for statements like creating or dropping tables.
- Set the `sql` parameter to the SQL statement and the `stream` parameter to the data you want to send to Db2. Use this when inserting data.

This application will show both ways. It executes SQL statements that: 
- Drop the Db2 table, if exists.
- Create a new table in a Db2 database.
- Insert some rows into the table.
- Select all rows from a table.

### Define the SQL statements and table name

In [13]:
table_name = 'RUN_SAMPLE_DEMO'

# SQL statements
sql_drop   = 'DROP TABLE ' + table_name
sql_create = 'CREATE TABLE ' + table_name + ' (ID INT, NAME CHAR(30), AGE INT)'
sql_insert = 'INSERT INTO ' + table_name + ' (ID, NAME, AGE) VALUES (? , ?, ?)'
sql_select = 'SELECT * FROM ' + table_name

## <a name="drop"> </a> 2.1. Create the table


In the following step the `topo.source` creates a `Stream` containing the two SQL statements to drop and create the table. 

`db.run_statement` executes the two statements in the input stream, so it will drop the table and create a new table.


In [14]:
# The streamToCreateTable is a Stream containing the two SQL statements: sql_drop and sql_create
streamToCreateTable = topo.source([sql_drop, sql_create]).as_string()
# drop the table if exist and create a new table in database
db.run_statement(stream=streamToCreateTable, credentials=db2credentials, name="CREATE_TABLE",)


## <a name="insert"> </a> 2.2. Insert streaming data into the table

Next, we generate a stream of data and insert it into the table we created.

The function `generate_data()` generates some data with schema `(ID INTEGER, NAME STRING, AGE INTEGER)` that will be inserted into the database..

Before it can be inserted in the database, we have to change the [schema](https://streamsxtopology.readthedocs.io/en/stable/streamsx.topology.schema.html) of the input data `Stream` to the [StreamsSchema](https://streamsxtopology.readthedocs.io/en/stable/streamsx.topology.schema.html#streamsx.topology.schema.StreamSchema) type, which is the format accepted by the `run_statement` function.  See the [list of mappings from Python types to StreamSchema types](https://streamsxtopology.readthedocs.io/en/stable/streamsx.topology.schema.html#streamsx.topology.schema.StreamSchema)


The `genData` `Stream` contains the data produced by the `generate_data()` function.

We again use `db.run_statement` but in the following step, it uses `genData` as input stream and the predefined `sql_insert` variable as the SQL statement.


In [15]:
import random
import time

# generates some data with schema (ID, NAME, AGE)
def generate_data():
    counter = 0
    while True:
        #yield a random id, name and age
        counter = counter +1 
        yield  {"NAME": "Name_" + str(random.randint(0,500)), "ID": counter, "AGE": random.randint(10,99)}
        time.sleep(0.10)

# convert it to SPL schema for the database operator run_statement
tuple_schema = StreamSchema("tuple<int64 ID, rstring NAME, int32 AGE>")
# Generates data for a stream of three attributes. Each attribute maps to a column using the same name of the Db2 database table.
genData = topo.source(generate_data, name="GeneratedData").map(lambda tpl: (tpl["ID"], tpl["NAME"], tpl["AGE"]),
                    schema=tuple_schema)


# insert generated rows into table
insertResults= db.run_statement(name="INSERT", stream=genData, sql=sql_insert, sql_params="ID, NAME, AGE" ,credentials = db2credentials)

## <a name="select"> </a> 2.3. Retrieve data from the table
In this step the `run_statement` runs the SQL statement `"SELECT * FROM RUN_SAMPLE_DEMO"` and returns the results in tuple schema `tuple<int64 ID, rstring NAME, int32 AGE>` .


In [16]:
# select all rows from table
selectResults= db.run_statement(name="SELECT", schema='tuple<int64 ID, rstring NAME, int32 AGE>', stream=genData, sql=sql_select, credentials = db2credentials)
selectResults.print()

# create a view to check retrieving data from a table
selectView = selectResults.view(name="selectRecords", description="Sample of selected records")


# <a name="submit"> </a> 3. Submit the application

A running Streams application is called a *job*. This next cell submits the application for execution and prints the resulting job id.

In [17]:
from streamsx.topology import context

# Disable SSL certificate verification if necessary
cfg[context.ConfigParams.SSL_VERIFY] = False
# submit the topology 'topo'
submission_result = context.submit ("DISTRIBUTED", topo, config = cfg)

# The submission_result object contains information about the running application, or job
if submission_result.job:
    streams_job = submission_result.job
    print ("JobId: ", streams_job.id , "\nJob name: ", streams_job.name)

IntProgress(value=0, bar_style='info', description='Initializing', max=10, style=ProgressStyle(description_wid…

Insecure host connections enabled.
Insecure host connections enabled.
Insecure host connections enabled.
/user-home/_global_/python-3/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


JobId:  59 
Job name:  StreamsTutorialandTestbed::DatabaseSample_59


# <a name="view"> </a> 4. Use the View to access data from the job

Now that the job is started, use the View object you have already created to start retrieving data from a table in database.

In [ ]:
# Connect to the view and display the selected data
queue = selectView.start_data_fetch()
try:
    for val in range(20):
        print(queue.get())    
finally:
    selectView.stop_data_fetch()

# <a name="status"> </a> 5. See job status

You can view job status and logs by going to My Instances > Jobs. Find your job based on the id printed above. Retrieve job logs using the "Download logs" action from the job's context menu.

To view other information about the job such as detailed metrics, access the graph. Go to My **Instances** > **Jobs**. Select "View graph" action for the running job.



# <a name="cancel"></a> 6. Cancel the job

This cell generates a widget you can use to cancel the job.


In [ ]:
# cancel the job in the IBM Streams service
submission_result.cancel_job_button()

You can also interact with the job through the Job object returned from 
`submission_result.job`.

For example, use  `job.cancel()` to cancel the running job directly.

## Summary

We created an application which connects to Db2 database, dropped a table, created a table, inserted some rows into table and reads the rows.

After submitting the application to the Streams service, we checked the application logs to see the progress.

It is also possible to check the contents of the test table on Db2 console with the following command.

      db2 "SELECT * FROM RUN_SAMPLE_DEMO"
